In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
# 1. Data Preparation and Loading
# create **known** parameters
weight = 0.7
bias = 0.3

# create 
start = 0
end = 1
steps = 0.02
X = torch.arange(start, end, steps).unsqueeze(dim=1) # add 1 dims

y = weight * X + bias
X[:10]
y[:10]

tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])

In [3]:
len(X), len(y)

(50, 50)

In [4]:
## Spliting data into training and test sets    
# traing, validation and test set

# creating training set
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [5]:
#a# Visualize, Visualize, Visualize
import matplotlib.pyplot as plt

In [7]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14})
  plt.show()

In [8]:
plot_predictions()

: 

In [ ]:
## Build a model
# Create linear regression model class
## building classes that can use the following link:
https://realpython.com/python-classes/


In [ ]:
# what model does:
# start with random weights and bias
# look at the training data and adjust weights and bias to better represent the data

# through two main algorithms:
# 1. Gradient Descent
# 2. Backpropagation

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad = True,
                                                dtype = torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad = True,
                                             dtype = torch.float))
        # Forward method to define the computation in the model
        def forward(self,x: torch.Tensor) -> torch.Tensor: # x is input data
            return self.weights * x + self.bias
        
    

In [9]:
import math
class circle:
    def __init__(self, radius):
        self.radius = radius
    def calculate_area(self):
        return math.pi * self.radius ** 2
    


In [11]:
circle1 = circle(5)
circle2 = circle(10)

print(circle1.calculate_area(), circle2.calculate_area())

78.53981633974483 314.1592653589793
